In [8]:
import gradio as gr
from openai import OpenAI

from math import exp

In [14]:
def classify_text(model_name,classes,text):
    client = OpenAI()
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that classifies text into predefined categories."},
            {"role": "user", "content": f"Classify the following text into one of these categories: {', '.join(classes)}.\n\nText: {text}"}
        ],
        logprobs=True,
        top_p = 0.5,
        n=10
    )
    probs = {
        completion.message.content : exp(sum([lp.logprob for lp  in completion.logprobs.content]))
        for completion in response.choices
        if completion.message.content in classes
    }
    for c in classes:
        if c not in probs:
            probs[c] =0
    print(probs)
    return probs
    

In [10]:
classes_list = []

In [11]:
def add_class(new_class):
    if new_class and new_class not in classes_list:
        classes_list.append(new_class)
    return gr.List([classes_list],label="Classes")
def remove_class(class_to_remove):
    if class_to_remove in classes_list:
        classes_list.remove(class_to_remove)
    return gr.List([classes_list],label="Classes")


In [15]:
classes_list = []
with gr.Blocks() as demo:
    gr.Markdown("## Text Classification App")
    model_name = gr.Dropdown(label="Model Name", choices=["gpt-3.5-turbo", "gpt-4o-mini"], value="gpt-3.5-turbo")
    text = gr.Textbox(label="Text to be analized")

    with gr.Row():
        with gr.Column():
            classes = gr.List([],label="Classes")
        with gr.Column():
            with gr.Row():
                new_class = gr.Textbox(label="New Class")
                add_class_btn = gr.Button("Add Class")
            with gr.Row():
                remove_class_input = gr.Dropdown(label="Remove Class", choices=[], value=None)
                remove_class_btn = gr.Button("Remove Class")

    add_class_btn.click(add_class, [new_class], [classes])
    add_class_btn.click(lambda: gr.update(choices=classes_list), None, remove_class_input)
    add_class_btn.click(lambda: "", None, new_class)
    
    remove_class_btn.click(remove_class, [remove_class_input], [classes])
    remove_class_btn.click(lambda: gr.update(choices=classes_list), None, remove_class_input)
   
    output_probabilities = gr.Label(label="Probabilities")
    

    classify_btn = gr.Button("Classify")
    classify_btn.click(lambda m,t: classify_text(m,classes_list,t) , [model_name,text], [output_probabilities])

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


{'Negatív': 0.9358208464187076, 'Pozitív': 0}
{'Pozitív': 0, 'Negatív': 0}
{'Negatív': 0.9358208464187076, 'Pozitív': 0}
